In [1]:
!pip install gensim scikit-learn xgboost catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_vers

In [6]:
!pip install --upgrade --force-reinstall numpy pandas scikit-learn xgboost catboost gensim --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 133.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 550.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np
from google.colab import files


In [2]:
from google.colab import drive

# 드라이브 마운트
drive.mount('/content/drive')

# 경로 설정
file_path = '/content/drive/MyDrive/movie/filtered_over_100k_utf8sig.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 🎬 전처리
df['actor_list'] = df['actors'].fillna('').apply(lambda x: x.split('|'))
df['director_list'] = df['directors'].fillna('').apply(lambda x: [x.strip()])


In [5]:
# 🧠 Word2Vec 학습 (20 epochs)
actor_corpus = df['actor_list'].tolist()
w2v_actor = Word2Vec(vector_size=8, window=3, min_count=1, sg=1, seed=42)
w2v_actor.build_vocab(actor_corpus)
w2v_actor.train(actor_corpus, total_examples=w2v_actor.corpus_count, epochs=20)

director_corpus = df['director_list'].tolist()
w2v_director = Word2Vec(vector_size=4, window=3, min_count=1, sg=1, seed=42)
w2v_director.build_vocab(director_corpus)
w2v_director.train(director_corpus, total_examples=w2v_director.corpus_count, epochs=20)

(18244, 19300)

In [6]:
def get_mean_vector(entities, model):
    vectors = [model.wv[entity] for entity in entities if entity in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

In [7]:
df['actor_vec'] = df['actor_list'].apply(lambda x: get_mean_vector(x, w2v_actor))
df['director_vec'] = df['director_list'].apply(lambda x: get_mean_vector(x, w2v_director))

In [11]:
# 🎭 장르 원핫 인코딩
df['genre_list'] = df['genres'].fillna('').apply(lambda x: x.split('|'))
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre_list'])
print(genre_encoded[1])

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [16]:
# 🌎 국가 가중치 인코딩
nation_avg = df.groupby('nations')['audiCnt'].mean()
nation_weights = (nation_avg / nation_avg.max()).to_dict()
df['nation_weight_encoded'] = df['nations'].apply(lambda x: nation_weights.get(x, 0.0))
print(df['nation_weight_encoded'].iloc[50])

0.3879942252514979


In [17]:
# ✅ X 생성
X = np.vstack([
    np.concatenate([a, d, g, [n]])
    for a, d, g, n in zip(df['actor_vec'], df['director_vec'], genre_encoded, df['nation_weight_encoded'])
])


In [18]:
# 🎯 타겟값: log1p(audiCnt)
y = np.log1p(df['audiCnt'].values)


In [19]:
# 📊 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# ✅ 모델 정의 (CatBoost 포함)
models = {
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42),
    "CatBoost": CatBoostRegressor(n_estimators=100, verbose=0, random_state=42),
    "MLP": MLPRegressor(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
}

In [23]:
from sklearn.metrics import mean_squared_error, r2_score

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_true), np.expm1(y_pred)))

def r2(y_true, y_pred):
    return r2_score(np.expm1(y_true), np.expm1(y_pred))

In [24]:
# ✅ 학습 및 평가 (RMSE + R²)
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score_rmse = rmse(y_test, y_pred)
    score_r2 = r2(y_test, y_pred)
    print(f"✅ {name:<12} RMSE: {score_rmse:,.2f} | R²: {score_r2:.4f}")


✅ RandomForest RMSE: 2,096,936.65 | R²: 0.0292
✅ XGBoost      RMSE: 2,245,983.47 | R²: -0.1137
✅ CatBoost     RMSE: 2,134,597.49 | R²: -0.0060
✅ MLP          RMSE: 2,091,409.26 | R²: 0.0343


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
